# Kütüphanelerin indirilmesi ve verinin yüklenmesi

In [ ]:
pip install numpy==1.26.4

In [11]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import nltk

nltk.download('punkt') # nltk'nın punkt modülü, metin parçalama (tokenization) işlemi için kullanılır.
nltk.download('wordnet') # WordNet adı verilen bir dilbilgisi veritabanıdır. Sözcüklerin eş anlamıları, karşıt anlamlıları ve sözcükler arasındaki ilişkiler gibi dilbilgisi bilgilerini içerir.

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

import json
import pickle

intents_file = open('intents.json').read()
intents = json.loads(intents_file)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\erend\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\erend\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Veri Ön İşleme

In [2]:
words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        #Her bir kelimeyi tokenize ediyoruz.
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        
        #documents listesine belge ekleme
        documents.append((word, intent['tag']))
        
        #sınıf listemize ekliyoruz
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

In [3]:
# Her bir kelimeyi küçük hale getirip Lemmatize etme ve böylelikle aynı öğeleri kaldırma

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))

# Classları sıralayalım
classes = sorted(list(set(classes)))

print(len(documents), 'documents')

# classes = intents
print (len(classes), "classes", classes)

# word = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
87 unique lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


# Training ve Test Verilerini Oluşturmak

In [5]:
training = []
# çıktılar için boş bir array oluşturdum
output_empty = [0] * len(classes)
# Eğitim Seti, Her cümle için Kelime bag'i
for doc in documents:
    # Bag'i yani kelimelerimizi içerisine atacağımız listeyi oluşturdum.
    bag = []
    # Modelimiz için tokenization işlemi yaptığım kelimelerin listesi
    word_patterns = doc[0]
    # Her bir kelimeyi Lemmatize ettim
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # Eğer geçerli kalıpta kelime bulunursa 1 ile bag dizisi içerisine yaz.
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    # çıktı her etiket için '0' ve mevcut etiket için '1'dir (her model için)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# özellikleri karıştırdım ve numpy dizisi oluşturdum
random.shuffle(training) # training listemizin içerisindeki verileri karıştırdık(Öğelerin sıralaması rastgele değiştirilir). Bu her bir eğitim döngüsünde modelin farklı sırarlarda örnekler sunarak modelimizin daha iyi eğitilmesine yardımcı oluyor ve sadece belirli bir düzene göre öğrenerek ezberlemesini engeller. 
arrayli_training = np.array(training, dtype="object") #Burada ilk hata aldım fakat ardından dtype'ı eklediğimde bir hata almadım.

# Eğitim ve Test listelerini oluşturdum. X- Patternsler, Y- intentsler (Kullanıcının istekleri)
train_x = list(arrayli_training[:,0])
train_y = list(arrayli_training[:,1])
print("Training data is created")

Training data is created


In [12]:
# deep neural networds model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = tf.keras.optimizers.legacy.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#Training and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model is created")

Epoch 1/200


C:\Users\erend\anaconda3\lib\site-packages\keras\src\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


10/10 [==============================] - 1s 7ms/step - loss: 2.2482 - accuracy: 0.0426
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1196 - accuracy: 0.2553
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0869 - accuracy: 0.2553
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9678 - accuracy: 0.3191
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8627 - accuracy: 0.3617
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 1.6958 - accuracy: 0.4468
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7116 - accuracy: 0.4894
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3520 - accuracy: 0.5957
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3469 - accuracy: 0.5957
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.2084 - accuracy: 0.6809
Epoch 11/200
10/10 [=

10/10 [==============================] - 0s 2ms/step - loss: 0.0336 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0769 - accuracy: 0.9787
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0750 - accuracy: 0.9787
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1182 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0907 - accuracy: 0.9574
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0374 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0392 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0808 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0577 - accuracy: 1.0000
Epoch 92/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0639 - accuracy: 0.9787
Epoch 93/200


10/10 [==============================] - 0s 2ms/step - loss: 0.0153 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0136 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0172 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0202 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0433 - accuracy: 0.9574
Epoch 169/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0204 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0116 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0235 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0186 - accuracy: 1.0000
Epoch 173/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0543 - accuracy: 0.9787
Epoc

C:\Users\erend\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
